# LangChain v1 RAG Application Examples

This notebook demonstrates usage patterns for the refactored RAG application using **LangChain v1**.

## Contents
1. Setup and Initialization
2. Document Ingestion Patterns
3. RAG Chat with Agents
4. Advanced Agent Patterns
5. Retriever Customization
6. Conversation Memory
7. Structured Outputs
8. Error Handling and Observability

## 1. Setup and Initialization

In [1]:
# Add project root to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Verify imports
from acc_llamaindex.config import config
from acc_llamaindex.infrastructure.llm_providers.langchain_provider import get_llm, get_embeddings, reset_llm
from acc_llamaindex.infrastructure.db.chroma_client import chroma_client
from acc_llamaindex.application.ingest_documents_service.service import ingest_service
from acc_llamaindex.application.chat_service.service import chat_service
from acc_llamaindex.infrastructure.llm_providers import langchain_provider, anthropic_provider 
print("✓ All imports successful")

2025-10-17 20:04:28.700 | INFO     | acc_llamaindex.application.ingest_documents_service.service:__init__:38 - IngestDocumentsService initialized with documents_path: /Users/kevinknox/coding/acc-llamaindex/data/documents
2025-10-17 20:04:28.747 | INFO     | acc_llamaindex.application.chat_service.service:__init__:18 - ChatService initialized


✓ All imports successful


## 2. Document Ingestion Patterns

### Pattern 1: Basic Document Ingestion

In [2]:
# Initialize services
get_llm()
get_embeddings()
chroma_client.initialize()

print(f"Documents path: {config.documents_path}")
print(f"ChromaDB path: {config.chroma_db_path}")
print(f"Chunk size: {config.chunk_size}, overlap: {config.chunk_overlap}")

2025-10-17 20:04:30.768 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:_initialize_llm:61 - Initializing LLM with provider: openai
2025-10-17 20:04:30.938 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:28 - Initializing ChatOpenAI with model: gpt-5-nano-2025-08-07
2025-10-17 20:04:31.057 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:38 - ChatOpenAI initialized successfully
2025-10-17 20:04:31.058 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:_initialize_embeddings:92 - Initializing embeddings with provider: openai
2025-10-17 20:04:31.058 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_embeddings:50 - Initializing OpenAIEmbeddings with model: text-embedding-3-small
2025-10-17 20:04:31.079 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_embeddings:57 - OpenAIEmbeddings initiali

Documents path: /Users/kevinknox/coding/acc-llamaindex/data/documents
ChromaDB path: /Users/kevinknox/coding/acc-llamaindex/data/chroma_db
Chunk size: 1024, overlap: 200


In [3]:
# Ingest documents from default directory
result = ingest_service.ingest_documents_from_directory()

print(f"Success: {result.success}")
print(f"Documents processed: {result.documents_processed}")
print(f"Documents failed: {result.documents_failed}")
print(f"Message: {result.message}")
print(f"\nCollection stats: {result.collection_stats}")

2025-10-17 20:04:33.447 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:78 - Starting document ingestion from: /Users/kevinknox/coding/acc-llamaindex/data/documents
2025-10-17 20:04:33.451 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:91 - Found 4 documents to process
2025-10-17 20:04:33.451 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:102 - Loading document: langchain_intro.txt
2025-10-17 20:04:33.452 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:102 - Loading document: US Trade with sub_Saharan Africa 11162023_0.pdf
2025-10-17 20:04:33.515 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:102 - Loading document: IncoDocs-Global-Trade-Toolkit225.pdf
2025-10-17 20:04:34.468 | INFO     | acc_llamaindex

Success: True
Documents processed: 4
Documents failed: 0
Message: Successfully ingested 4 documents (120 chunks)

Collection stats: {'collection_name': 'documents', 'document_count': 20924, 'status': 'active'}


### Pattern 2: Ingest from Custom Directory

In [34]:
# Create custom directory with documents
import tempfile
import os

temp_dir = tempfile.mkdtemp()

# Create test documents
test_doc = os.path.join(temp_dir, "test.txt")
with open(test_doc, "w") as f:
    f.write("This is a test document about artificial intelligence and machine learning.")

# Ingest from custom directory
result = ingest_service.ingest_documents_from_directory(temp_dir)
print(f"Ingested {result.documents_processed} documents from custom directory")

# Cleanup
import shutil
shutil.rmtree(temp_dir)

2025-10-17 20:02:11.402 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:78 - Starting document ingestion from: /var/folders/zq/63gj0t315wl3ltbkp5_26d9m0000gn/T/tmpq2rxy3ed
2025-10-17 20:02:11.403 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:91 - Found 1 documents to process
2025-10-17 20:02:11.403 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:102 - Loading document: test.txt
2025-10-17 20:02:11.404 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:117 - Created 1 text chunks from 1 documents
2025-10-17 20:02:12.545 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_documents_from_directory:123 - Successfully added 1 chunks to vector store


Ingested 1 documents from custom directory


### Pattern 3: Ingest Single File

In [5]:
# Ingest a single file
single_file = "../data/documents/txt/langchain_intro.txt"
result = ingest_service.ingest_single_file(single_file)

print(f"Success: {result.success}")
print(f"Message: {result.message}")

2025-10-17 20:05:05.970 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_single_file:158 - Ingesting single file: ../data/documents/txt/langchain_intro.txt
2025-10-17 20:05:05.971 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_single_file:167 - Created 1 text chunks from langchain_intro.txt
2025-10-17 20:05:13.623 | INFO     | acc_llamaindex.application.ingest_documents_service.service:ingest_single_file:173 - Successfully indexed file: langchain_intro.txt


Success: True
Message: Successfully ingested langchain_intro.txt (1 chunks)


## 3. RAG Chat with Agents

### Pattern 1: Basic Chat

In [6]:
# Switch to Anthropic
config.llm_provider = "openai"
reset_llm() 
llm = get_llm()  # Now using Claude
 
# Initialize chat service
chat_service.initialize() 
 
# Simple chat query
response = chat_service.chat("What is AGOA?")

print(f"Success: {response['success']}")
print(f"\nResponse:\n{response['response']}")

2025-10-17 20:05:29.196 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:reset_llm:48 - LLM instance reset
2025-10-17 20:05:29.197 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:_initialize_llm:61 - Initializing LLM with provider: openai
2025-10-17 20:05:29.197 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:28 - Initializing ChatOpenAI with model: gpt-5-nano-2025-08-07
2025-10-17 20:05:29.198 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:38 - ChatOpenAI initialized successfully
2025-10-17 20:05:29.198 | INFO     | acc_llamaindex.application.chat_service.service:initialize:23 - Initializing ChatService...
2025-10-17 20:05:29.206 | INFO     | acc_llamaindex.application.chat_service.service:initialize:67 - ChatService initialized successfully
2025-10-17 20:05:29.207 | INFO     | acc_llamaindex.application.chat_service.service:chat:88 - Processing chat 

Success: True

Response:
Short answer: AGOA stands for the Africa Growth and Opportunity Act. It is a U.S. trade act enacted in 2000 to provide enhanced access to the U.S. market for eligible sub-Saharan African countries, including duty-free or preferential treatment for many products and other trade benefits intended to support economic development.

What I can verify from the knowledge base you provided:
- The documents reference AGOA in connection with section 502 of the 1974 Act and note that the President must monitor and annually review progress under section 506A. They do not provide a full definition of AGOA within those excerpts.

If you’d like, I can fetch more sources that give a fuller definition and details (eligibility criteria, product coverage, renewal terms, etc.). Sources cited: Documents mentioning AGOA (relating to the 1974 Act) in your knowledge base.


### Pattern 2: Chat with Conversation History

In [7]:
# Start a conversation
conversation_history = []

# First message
response1 = chat_service.chat(
    "What are the key features of Incoterms?",
    conversation_history=conversation_history
)
print("User: What are the key features of Incoterms?")
print(f"Assistant: {response1['response'][:200]}...\n")

# Add to history
conversation_history.extend([
    {"role": "user", "content": "What are the key features of Incoterms?"},
    {"role": "assistant", "content": response1['response']}
])

# Follow-up message
response2 = chat_service.chat(
    "Can you explain more about Incoterms?",
    conversation_history=conversation_history
)
print("User: Can you explain more about Incoterms?")
print(f"Assistant: {response2['response'][:200]}...")

2025-10-17 20:06:53.141 | INFO     | acc_llamaindex.application.chat_service.service:chat:88 - Processing chat message: What are the key features of Incoterms?...
2025-10-17 20:07:30.337 | INFO     | acc_llamaindex.application.chat_service.service:chat:103 - Chat response generated successfully
2025-10-17 20:07:30.338 | INFO     | acc_llamaindex.application.chat_service.service:chat:88 - Processing chat message: Can you explain more about Incoterms?...


User: What are the key features of Incoterms?
Assistant: Here’s a concise overview of the key features of Incoterms (Incoterms 2020), as described in the Incoterms 2020 Rules Responsibility Quick Reference Guide in the knowledge base:

- Allocation of respo...



2025-10-17 20:07:54.393 | INFO     | acc_llamaindex.application.chat_service.service:chat:103 - Chat response generated successfully


User: Can you explain more about Incoterms?
Assistant: Here’s a more in-depth, but still high-level, explanation of Incoterms and how they work.

What Incoterms are
- Incoterms are a set of standardized trade terms published by the International Chamber o...


## 4. Advanced Agent Patterns

### Pattern 1: Direct Agent Creation with Custom Tools

In [9]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from datetime import datetime


# Create custom tools
@tool
def get_current_time() -> str:
    """Get the current time in a human-readable format."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def search_documents(query: str) -> str:
    """Search the document knowledge base for relevant information."""
    vector_store = chroma_client.get_vector_store()
    docs = vector_store.similarity_search(query, k=3)
    if not docs:
        return "No relevant documents found."
    return "\n\n".join([doc.page_content for doc in docs])

# Create agent with multiple tools
config.llm_provider = "openai"
reset_llm() 
llm = get_llm()
agent = create_agent(
    model=llm,
    tools=[get_current_time, search_documents],
    system_prompt="You are a helpful assistant with access to document search and time utilities."
)
 
# Test the agent
response = agent.invoke({
    "messages": [{"role": "user", "content": "What time is it and what do the documents say about African free trade?"}]
})

print("Agent response:")
for msg in response["messages"]:
    if hasattr(msg, 'content') and msg.content:
        print(f"{msg.__class__.__name__}: {msg.content[:200]}...")

2025-10-17 12:16:06.308 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:reset_llm:48 - LLM instance reset
2025-10-17 12:16:06.308 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:_initialize_llm:61 - Initializing LLM with provider: openai
2025-10-17 12:16:06.309 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:28 - Initializing ChatOpenAI with model: gpt-5-nano-2025-08-07
2025-10-17 12:16:06.309 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:38 - ChatOpenAI initialized successfully


Agent response:
HumanMessage: What time is it and what do the documents say about African free trade?...
ToolMessage: 2025-10-17 12:16:10...
ToolMessage: Subcommittee strongly prefers 
electronic submissions made through 
the Federal eRulemaking Portal: https:// 
www.regulations.gov (Regulations.gov). 
Follow the instructions for submitting 
written co...
AIMessage: Current time: 12:16:10 on October 17, 2025.

Documents found:
- The materials returned focus on AGOA (the African Growth and Opportunity Act), not specifically on AfCFTA (the African Continental Free ...


### Pattern 2: Agent with Dynamic Model Selection

In [ ]:
from langchain.agents.middleware import wrap_model_call, ModelRequest
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Create different models for different complexity
basic_model = ChatGroq(model="openai/gpt-oss-120b", temperature=0.3)
advanced_model = ChatOpenAI(model="gpt-5-mini-2025-08-07", temperature=0.7)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler):
    """Select model based on message count."""
    message_count = len(request.state["messages"])
    
    # Use advanced model for complex conversations
    if message_count > 5:
        print(f"Using advanced model (message count: {message_count})")
        request.model = advanced_model
    else:
        print(f"Using basic model (message count: {message_count})")
        request.model = basic_model
    
    return handler(request)

# Create agent with dynamic model selection
agent = create_agent(
    model=basic_model,
    tools=[search_documents],
    middleware=[dynamic_model_selection]
)

# Test with simple query
response = agent.invoke({"messages": [{"role": "user", "content": "Hello!"}]})
print("\nSimple query completed")

Using basic model (message count: 1)

Simple query completed


## 5. Retriever Customization

### Pattern 1: Custom Retriever with Score Threshold

In [9]:

# Get vector store
vector_store = chroma_client.get_vector_store() 

# Similarity search with scores
query = "What are all the incoterms in the Incoterms 2022?"
results = vector_store.similarity_search_with_score(query, k=5)

print(f"Query: {query}\n")
for i, (doc, score) in enumerate(results, 1):
    print(f"Result {i} (score: {score:.4f}):")
    print(f"{doc.page_content[:350]}...\n")

Query: What are all the incoterms in the Incoterms 2022?

Result 1 (score: 0.6255):
to, which may occur when it is being used to confirm complex commercial agreements.
All parties must make it clear in contracts which Incoterms® version is being referred to in order to avoid any
misunderstanding. Different trading partners will incorporate Incoterms® into contracts at different times.
It is imperative that you check existing contr...

Result 2 (score: 0.6255):
to, which may occur when it is being used to confirm complex commercial agreements.
All parties must make it clear in contracts which Incoterms® version is being referred to in order to avoid any
misunderstanding. Different trading partners will incorporate Incoterms® into contracts at different times.
It is imperative that you check existing contr...

Result 3 (score: 0.6255):
to, which may occur when it is being used to confirm complex commercial agreements.
All parties must make it clear in contracts which Incoterms® version i

### Pattern 2: Multi-Query Retrieval

In [10]:
# Multiple related queries
queries = [
    "What are incoterms?",
    "How do incoterms affect shipping?",
    "What are the benefits of incoterms?"
]

all_results = []
for query in queries:
    results = vector_store.similarity_search(query, k=2)
    all_results.extend(results)

# Deduplicate based on content
unique_docs = {}
for doc in all_results:
    unique_docs[doc.page_content[:100]] = doc

print(f"Retrieved {len(unique_docs)} unique documents from {len(queries)} queries")
for i, doc in enumerate(list(unique_docs.values())[:3], 1):
    print(f"\nDocument {i}:")
    print(f"{doc.page_content[:150]}...")

Retrieved 1 unique documents from 3 queries

Document 1:
Put simply, Incoterms® are the selling terms that the buyer and seller of goods both agrees to.  The Incoterm®
clearly states which tasks, costs and r...


## 6. Conversation Memory

### Pattern 1: Persistent Conversation with Checkpointer

In [21]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from acc_llamaindex.config import config

# Create agent with memory
checkpointer = InMemorySaver() 

#toggle between providers
config.llm_provider = "groq"
reset_llm() 
llm = get_llm() 

agent = create_agent(
    model=get_llm(),
    tools=[search_documents],
    checkpointer=checkpointer,
    system_prompt="You are a helpful assistant. Remember the conversation context and limit answers to 5 words."
)

# Conversation thread
config: RunnableConfig = {"configurable": {"thread_id": "user-123"}}

# First message
response1 = agent.invoke(
    {"messages": [{"role": "user", "content": "My name is Alice"}]},
    config
)
print("User: My name is Alice")
print(f"Assistant: {[m for m in response1['messages'] if hasattr(m, 'content')][-1].content[:100]}...\n")

# Second message (should remember name)
response2 = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    config
)
print("User: What's my name?")
print(f"Assistant: {[m for m in response2['messages'] if hasattr(m, 'content')][-1].content}")

2025-10-17 18:19:32.567 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:reset_llm:48 - LLM instance reset
2025-10-17 18:19:32.567 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:_initialize_llm:61 - Initializing LLM with provider: groq
2025-10-17 18:19:32.568 | INFO     | acc_llamaindex.infrastructure.llm_providers.groq_provider:initialize_llm:31 - Initializing ChatGroq with model: openai/gpt-oss-120b
2025-10-17 18:19:32.597 | INFO     | acc_llamaindex.infrastructure.llm_providers.groq_provider:initialize_llm:41 - ChatGroq initialized successfully


User: My name is Alice
Assistant: Nice to meet you, Alice....

User: What's my name?
Assistant: Your name is Alice.


## 7. Structured Outputs

### Pattern 1: Extract Structured Data from Documents

In [12]:
from pydantic import BaseModel, Field

# Define schema
class DocumentSummary(BaseModel):
    """Summary of document content."""
    title: str = Field(description="Main topic or title")
    key_points: list[str] = Field(description="List of key points mentioned")
    category: str = Field(description="Document category (e.g., technical, guide, reference)")

# reset llm
config.llm_provider="openai"
reset_llm()
llm = get_llm()

# Create a structured output LLM (without agent)
structured_llm = llm.with_structured_output(DocumentSummary)

# First, search for relevant documents
vector_store = chroma_client.get_vector_store()
docs = vector_store.similarity_search("Incoterms", k=5)
context = "\n\n".join([f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

# Then use structured output to summarize
prompt = f"""Based on the following documents, provide a structured summary about Incoterms:

{context}

Provide a title, category, and key points."""

summary = structured_llm.invoke(prompt)

print(f"Title: {summary.title}")
print(f"Category: {summary.category}")
print(f"\nKey Points:")
for i, point in enumerate(summary.key_points, 1):
    print(f"{i}. {point}")

2025-10-17 20:12:09.602 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:reset_llm:48 - LLM instance reset
2025-10-17 20:12:09.602 | INFO     | acc_llamaindex.infrastructure.llm_providers.langchain_provider:_initialize_llm:61 - Initializing LLM with provider: openai
2025-10-17 20:12:09.602 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:28 - Initializing ChatOpenAI with model: gpt-5-nano-2025-08-07
2025-10-17 20:12:09.603 | INFO     | acc_llamaindex.infrastructure.llm_providers.openai_provider:initialize_llm:38 - ChatOpenAI initialized successfully


Title: Incoterms 2020 Rules Responsibility Quick Reference Guide
Category: Technical reference / Guide

Key Points:
1. Purpose: Incoterms define the allocation of responsibilities, costs and risks between seller and buyer for international shipments.
2. Scope: The 2020 rules comprise 11 terms organized into four groups: E (EXW), F (FCA, FAS, FOB), C (CFR, CIF, CPT, CIP), and D (DAP, DPU, DDP).
3. Risk transfer: The point at which risk passes from seller to buyer varies by term (e.g., EXW at seller’s premises; FOB when goods pass the ship’s rail; DAP/DPU when delivered at the named place; DDP at import or delivery).
4. Cost allocation: Primary cost responsibilities (freight, insurance, export/import formalities, duties) differ by term (e.g., CFR/CIF: seller pays main carriage; CIF also requires minimum insurance; EXW: buyer bears most costs).
5. Key changes in 2020: DPU replaces the former DAT term (Delivered at Place Unloaded); DAP and DDP remain; CIP and CIF specify insurance requirem

## 8. Error Handling and Observability

### Pattern 1: Graceful Error Handling

In [ ]:
from langchain.agents.middleware import wrap_tool_call
from langchain_core.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Catch and handle tool execution errors."""
    try:
        return handler(request)
    except Exception as e:
        print(f"Tool error caught: {str(e)[:100]}")
        return ToolMessage(
            content=f"Tool execution failed. Please try rephrasing your request.",
            tool_call_id=request.tool_call["id"]
        )

# Create agent with error handling
agent = create_agent(
    model=get_llm(),
    tools=[search_documents],
    middleware=[handle_tool_errors]
)

# Test with query
response = agent.invoke({"messages": [{"role": "user", "content": "Search for information"}]})
print("Agent handled potential errors gracefully")

### Pattern 2: LangSmith Tracing

In [ ]:
import os
from langsmith import traceable

# Check if LangSmith is configured
if os.getenv("LANGCHAIN_API_KEY"):
    print("LangSmith tracing is enabled")
    print(f"Project: {os.getenv('LANGCHAIN_PROJECT', 'default')}")
    
    @traceable(run_type="chain", name="custom_rag_chain")
    def custom_rag_chain(query: str) -> str:
        """Custom RAG chain with tracing."""
        # Retrieve documents
        vector_store = chroma_client.get_vector_store()
        docs = vector_store.similarity_search(query, k=3)
        
        # Generate response
        context = "\n\n".join([doc.page_content for doc in docs])
        llm = get_llm()
        response = llm.invoke(
            f"Based on this context:\n{context}\n\nAnswer: {query}"
        )
        return response.content
    
    # Test with tracing
    result = custom_rag_chain("What are the benefits of RAG?")
    print(f"\nResponse: {result[:200]}...")
    print("\n✓ Check LangSmith for full trace details")
else:
    print("LangSmith not configured. Set LANGCHAIN_API_KEY to enable tracing.")

## Summary

This notebook demonstrated:

1. **Document Ingestion**: Multiple patterns for loading documents into the vector store
2. **RAG Chat**: Basic and conversational chat patterns
3. **Advanced Agents**: Custom tools, dynamic model selection, and middleware
4. **Retrievers**: Custom retrieval strategies and multi-query patterns
5. **Memory**: Conversation persistence with checkpointers
6. **Structured Outputs**: Extracting validated data from LLM responses
7. **Error Handling**: Graceful degradation and tool error management
8. **Observability**: LangSmith tracing integration

## Next Steps

- Experiment with different embedding models
- Try various chunk sizes and overlaps
- Implement reranking for better retrieval
- Add evaluation metrics (faithfulness, relevance, etc.)
- Explore LangGraph for complex workflows
- Add streaming responses for better UX